## 1. vector database 선택 및 설정
### 1) Pinecone
### 2) Chroma 
### 3) Weaviate 
### 4) Qdrant
### 5) Faiss
### 6) Milvus

https://meetcody.ai/ko/blog/2024%EB%85%84%EC%97%90-%EC%8B%9C%EB%8F%84%ED%95%B4-%EB%B3%BC-%EB%A7%8C%ED%95%9C-%EC%83%81%EC%9C%84-5%EA%B0%80%EC%A7%80-%EB%B2%A1%ED%84%B0-%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B2%A0%EC%9D%B4%EC%8A%A4/ 

# PostgreSQL Docker 이미지 실행
```terminal
docker run --name my_postgres -e POSTGRES_PASSWORD=aiv11011 -d -p 5432:5432 postgres
```

# 가상환경 생성, 활성화

```terminal
 python -m venv venv
 source venv/bin/activate
```

# 가상 환경 활성화된 상태에서 필요한 패키지 설치
```terminal
pip install psycopg2-binary sqlalchemy torch torchvision pillow numpy
```

# PostgreSQL 셸에서 데이터 확인
# 1. PostgreSQL 셸 접속
# 2. 데이터베이스 ㅡ사용
# 3. 데이터 조회
```terminal
docker exec -it my_postgres psql -U postgres
\c postgres
SELECT * FROM image_embeddings;
```

# 테이블 삭제
```terminal
DROP TABLE IF EXISTS image_embeddings;
```

# 테이블 다시 생성
```terminal
CREATE TABLE image_embeddings (
    id SERIAL PRIMARY KEY,
    image_path VARCHAR(255) UNIQUE NOT NULL,
    label VARCHAR(255) NOT NULL,
    embedding FLOAT[] NOT NULL
);
```
### Milvus 설치 및 설정 

# 1. Milvus 실행 위해 `docker-compose.yml` 파일 작성
#    해당 파일 있는 디렉토리에서 다음 명령어 실행 -> Milvus 시행
```terminal
docker-compose up -d
```

# 2. PyMilvus 설치
#    Milvus와 상호작용하기 위해 Python 클라이언트 라이브러리인 `pymilvus` 설치
```terminal
pip install pymilvus
```




# 설치 스크립트 다운로드
```terminal
curl -sfL https://raw.githubusercontent.com/milvus-io/milvus/master/scripts/standalone_embed.sh -o standalone_embed.sh
```

# 스크립트 실행 권한 부여
```terminal
chmod +x standalone_embed.sh
```

# Docker 컨테이너 시작
```terminal
bash standalone_embed.sh start
```

# Milvus 컨테이너 상태 확인 - 컨테이너가 정상적으로 실행 중인지 확인하려면 다음 명령어를 사용합니다.
```terminal
docker ps
```
#   실행 중인 컨테이너 목록에 milvus-standalone 컨테이너가 표시, STATUS가 healthy로 표시되면 정상적으로 실행된 것


# 의존성 install
```terminal
pip install -r requirements.txt
```






#
```terminal
docker run -d --name weaviate --env QUERY_DEFAULTS_LIMIT=25 --env CLUSTERS_PEERS_ACTION_BATCHING_WORKERS=4 --env CLUSTER_SLAVES_COUNT=2 --env AUTHENTICATION_ANONYMOUS_ACCESS_ENABLED=true --restart=unless-stopped --env PERSISTENCE_DATA_PATH="/var/lib/weaviate" --volume /var/lib/weaviate:/var/lib/weaviate --publish 8080:8080 semitechnologies/weaviate:1.19.2
```


### Pinecone - ConvNeXT, RegNet

In [ ]:
import os
import ssl
import time
import numpy as np
import torch
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
from pinecone import Pinecone, ServerlessSpec
from open_clip import create_model_and_transforms
from torchvision import transforms, models

# SSL 인증서 설정
import certifi
os.environ['SSL_CERT_FILE'] = certifi.where()
ssl._create_default_https_context = ssl._create_unverified_context

class CoCaImg2Vec():
    def __init__(self, model_name, pretrained, cuda=False):
        self.model, _, self.transform = create_model_and_transforms(model_name, pretrained=pretrained)
        self.device = torch.device("cuda" if cuda else "cpu")
        self.model = self.model.to(self.device)
        self.model.eval()

    def get_vec(self, img):
        image = self.transform(img).unsqueeze(0).to(self.device)
        with torch.no_grad():
            embedding = self.model.encode_image(image).cpu().numpy().flatten()
        return embedding

class Img2Vec():
    def __init__(self, model_name, cuda=False):
        self.device = torch.device("cuda" if cuda else "cpu")
        self.model = getattr(models, model_name)(pretrained=True).to(self.device)
        self.model.eval()
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def get_vec(self, img):
        image = self.transform(img).unsqueeze(0).to(self.device)
        with torch.no_grad():
            embedding = self.model(image)
        embedding_np = embedding.cpu().numpy().flatten()
        #print(f"Embedding dimension for {self.model.__class__.__name__}: {embedding_np.shape[0]}")
        return embedding_np

def load_images_from_folder(folder):
    images = []
    labels = []
    valid_image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']
    for class_folder_name in os.listdir(folder):
        class_folder_path = os.path.join(folder, class_folder_name)
        if not os.path.isdir(class_folder_path):
            continue
        for filename in os.listdir(class_folder_path):
            img_path = os.path.join(class_folder_path, filename)
            if os.path.splitext(filename)[1].lower() in valid_image_extensions:
                img = Image.open(img_path).convert('RGB')
                images.append(img)
                labels.append(class_folder_name)
    return images, labels

def create_pinecone_index(api_key, index_name, dimension):
    pc = Pinecone(api_key=api_key)
    if index_name not in pc.list_indexes().names():
        pc.create_index(
            name=index_name,
            dimension=dimension,
            metric='cosine',
            spec=ServerlessSpec(
                cloud='aws',
                region="us-east-1"
            )
        )
    return pc.Index(index_name)

def save_embeddings_to_pinecone(index, embeddings, labels, namespace):
    vectors = []
    for i, emb in enumerate(embeddings):
        vectors.append({
            "id": f"vec{i+1}",
            "values": emb.tolist(),
            "metadata": {"label": labels[i]}
        })

    index.upsert(
        vectors=vectors,
        namespace=namespace
    )
    
    return vectors

def classify_images_with_pinecone(model_dict, folder_path, api_key, index_name, namespace, cuda=False):
    images, labels = load_images_from_folder(folder_path)
    all_vectors = []

    for model_name, model in model_dict.items():
        print(f"Evaluating model {model_name}")
        img2vec = Img2Vec(model, cuda=cuda)
        if img2vec.model is None:
            continue

        start_time = time.time()
        embeddings = [img2vec.get_vec(img) for img in images]
        embeddings = [e for e in embeddings if e is not None]
        embeddings = np.array(embeddings)
        processing_time = (time.time() - start_time) / len(images)
        print(f"Processing Time per Image: {processing_time}")

        # Create Pinecone index
        index = create_pinecone_index(api_key, index_name, embeddings.shape[1])
        
        # Save embeddings to Pinecone
        vectors = save_embeddings_to_pinecone(index, embeddings, labels, namespace)
        all_vectors.extend(vectors)

    return all_vectors

def visualize_embeddings(vectors):
    print("Embedding vectors and their metadata:")
    for vector in vectors:
        print(f"ID: {vector['id']}, Label: {vector['metadata']['label']}, Vector: {vector['values'][:5]}...")

if __name__ == "__main__":
    folder_path = './data-gatter/train'
    api_key = 'f5001027-9bd4-4abb-8dd6-a2db16540ecc'
    index_name = 'quickstart'
    namespace = 'ns1'
    
    model_dict = {
        'ConvNeXt': 'convnext_base',
        'RegNet': 'regnet_y_16gf',
    }

    cuda = torch.cuda.is_available()

    vectors = classify_images_with_pinecone(model_dict, folder_path, api_key, index_name, namespace, cuda)
    visualize_embeddings(vectors)


# Docker를 사용하여 PostgreSQL 데이터베이스와 pgvector 확장을 설정하고, Python 스크립트를 실행하는 방법

# 1. Dockerfile 작성 
#    필요한 Python 환경 설정, 스크립트 실행하도록
#    requirements.txt 작성


# 2. docker-compose.yml 작성
#    PostgreSQL DB와 애플리케이션 동시에 실행할 수 있도록

# 3. PostgreSQL에 pgvector 확장 설치
#    pgvector 확장 설치 위해 PostgreSQL 컨테이너 시작된 후 초기화 스크립트를 실행하도록 설정.

# 4. 디렉토리 구조
```terminal
Issue2_vectorDB/
├── docker-compose.yml
├── Dockerfile
├── init-db.sh
├── Img2Vec.py
├── ImageEmbedding.py
├── postgresStore.py
├── postgresFind.py
└── requirements.txt

```

# 5. Docker Compose 실행
#    PostgreSQL DB와 애플리케이션 컨테이너 빌드하고 시작
#    PostgreSQL 컨테이너 - pgvector 확장 설치
#    애플리케이션 컨테이너 - `postgresStore.py` 스크립트 실행하여 이미지 임베딩 DB에 저장
```terminal
docker-compose up --build
```







# PostgreSQL 셸에서 데이터 확인
# 1. PostgreSQL 셸 접속
# 2. 데이터베이스 ㅡ사용
# 3. 데이터 조회
```terminal
docker exec -it my_postgres psql -U postgres
\c postgres
SELECT * FROM image_embeddings;
```

# 테이블 삭제
```terminal
DROP TABLE IF EXISTS image_embeddings;
```

# 테이블 다시 생성
```terminal
CREATE TABLE image_embeddings (
    id SERIAL PRIMARY KEY,
    image_path VARCHAR(255) UNIQUE NOT NULL,
    label VARCHAR(255) NOT NULL,
    embedding FLOAT[] NOT NULL
);
```
### Milvus 설치 및 설정 

# 1. Milvus 실행 위해 `docker-compose.yml` 파일 작성
#    해당 파일 있는 디렉토리에서 다음 명령어 실행 -> Milvus 시행
```terminal
docker-compose up -d
```

# 2. PyMilvus 설치
#    Milvus와 상호작용하기 위해 Python 클라이언트 라이브러리인 `pymilvus` 설치
```terminal
pip install pymilvus
```




# 설치 스크립트 다운로드
```terminal
curl -sfL https://raw.githubusercontent.com/milvus-io/milvus/master/scripts/standalone_embed.sh -o standalone_embed.sh
```

# 스크립트 실행 권한 부여
```terminal
chmod +x standalone_embed.sh
```

# Docker 컨테이너 시작
```terminal
bash standalone_embed.sh start
```

# Milvus 컨테이너 상태 확인 - 컨테이너가 정상적으로 실행 중인지 확인하려면 다음 명령어를 사용합니다.
```terminal
docker ps
```
#   실행 중인 컨테이너 목록에 milvus-standalone 컨테이너가 표시, STATUS가 healthy로 표시되면 정상적으로 실행된 것


# 의존성 install
```terminal
pip install -r requirements.txt
```






#
```terminal
docker run -d --name weaviate --env QUERY_DEFAULTS_LIMIT=25 --env CLUSTERS_PEERS_ACTION_BATCHING_WORKERS=4 --env CLUSTER_SLAVES_COUNT=2 --env AUTHENTICATION_ANONYMOUS_ACCESS_ENABLED=true --restart=unless-stopped --env PERSISTENCE_DATA_PATH="/var/lib/weaviate" --volume /var/lib/weaviate:/var/lib/weaviate --publish 8080:8080 semitechnologies/weaviate:1.19.2
```
